<a href="https://colab.research.google.com/github/ubaidillah-chem/fouling-ml/blob/main/01_dataset_compiler.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. Mount your drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

In [ ]:
# API and Viscosity calculator

def calc_api_and_viscosities(rho, watson_k):
  SG = rho / 999
  api = 141.5 / SG - 131.5

  if api > 4.78231 * watson_k - 50.3642:
    log_nu_100 = (
          4.39371
          - 1.94733 * watson_k
          + 0.12769 * watson_k**2
          + 0.00032629 * api**2
          - 0.0118246 * watson_k * api
          + ((0.1716 * watson_k**2 + 10.9943 * api + 0.0950663 * api**2 - 0.860218 * watson_k * api)
            / (api + 50.3642 - 4.78231 * watson_k))
      )
    log_nu_100 = min(log_nu_100, 6)
  else:
    log_nu_100 = 6
  nu_100 = 10**log_nu_100

  if api > 2.6296 * watson_k - 26.786:
    log_nu_210 = (
          -0.463634
          - 0.166532 * api
          + 0.000513447 * api**2
          - 0.00848995 * watson_k * api
          + ((0.080325 * watson_k + 1.24899 * api + 0.19768 * api**2)
            / (api + 26.786 - 2.6296 * watson_k))
      )
    log_nu_210 = min(log_nu_210, 6)
  else:
    log_nu_210 = 6
  nu_210 = 10**log_nu_210

  return api, nu_100, nu_210

# Load XGBOOST solubility model

import xgboost as xgb

model = xgb.Booster()
model.load_model("/content/drive/MyDrive/xgboost_solubility_model.json")

def predict_solubility(MW, weighted_avg_nbp, temp):
  press = 410 * 0.21
  input_data = np.array([[MW, weighted_avg_nbp, temp, press]])
  dmatrix = xgb.DMatrix(input_data)
  C_O2 = model.predict(dmatrix)[0]
  return C_O2


2. Make sure you have the excel file in My Drive and compile the dataset

In [ ]:
import pandas as pd
import numpy as np


sara_df = pd.read_excel("drive/MyDrive/Asomaning, 1997.xlsx", sheet_name="SARA and Elemental")
prop_df = pd.read_excel("drive/MyDrive/Asomaning, 1997.xlsx", sheet_name="Bulk Properties").iloc[1:]
prop_df[['API', 'Viscosity @ 100F', 'Viscosity @ 210F']] = prop_df.apply(lambda row: calc_api_and_viscosities(row['Density @ 60F'], row['Watson K']), axis=1, result_type='expand')

oil_props = []
for idx, row in prop_df.iterrows():
  if idx == 0:
    continue
  oil_type = row['Oil Type']
  prop = row.iloc[1:].to_numpy(dtype='float64')
  sara = sara_df[sara_df['Oil Type'] == oil_type].iloc[0, 1:].to_numpy(dtype='float64')

  oil_df = pd.read_excel("drive/MyDrive/Asomaning, 1997.xlsx", sheet_name=oil_type, skiprows=3).drop(columns=['Viscosity 1', 'Viscosity 2'])
  oil_df = oil_df.rename(columns={'Density': 'Density @ 60F'}).iloc[1:]
  oil_df[['API', 'Viscosity @ 100F', 'Viscosity @ 210F']] = oil_df.apply(lambda row: calc_api_and_viscosities(row['Density @ 60F'], row['Watson K']), axis=1, result_type='expand')
  oil_pcs = oil_df.iloc[:, 1:].to_numpy(dtype='float64').flatten()

  weighted_avg_nbp = (oil_df['NBP'] * oil_df['Molar Compositions ']).sum()
  oil_props.append([oil_type] + prop.tolist() + [weighted_avg_nbp] + sara.tolist() + oil_pcs.tolist())

sara_labels = sara_df.columns.to_list()[1:]
prop_labels = prop_df.columns.to_list()[1:]
pc_props = oil_df.columns.to_list()[1:]
pcs_labels = [f"PC{i}_{label}" for i in range(len(oil_df)) for label in pc_props]
oil_props = pd.DataFrame(oil_props, columns=['Oil Type'] + prop_labels + ['Weighted Avg NBP'] + sara_labels + pcs_labels)

test_matrix_df = pd.read_excel("drive/MyDrive/Asomaning, 1997.xlsx", sheet_name='Test Matrix').drop(columns=['Remarks   '])
runs = test_matrix_df['Run #  '].to_list()
test_matrix = []
for idx, row in test_matrix_df.iterrows():
  run = row['Run #  ']
  oil_type = row['Oil type']
  oil_prop = oil_props[oil_props['Oil Type'] == oil_type].iloc[0, 1:]
  thermo_data = row.iloc[2:].to_numpy(dtype='float64')

  test_matrix.append(np.concatenate(([run], oil_prop, thermo_data)))

oil_props_labels = oil_props.columns.to_list()[1:]
thermo_labels = test_matrix_df.columns.to_list()[2:]
test_matrix = pd.DataFrame(test_matrix, columns=['Run #'] + oil_props_labels + thermo_labels)

O2solubilities = test_matrix.apply(lambda row: predict_solubility(row['Mol Wt.'], row['Weighted Avg NBP'], row['Average Tb (°C) ']), axis=1)
test_matrix['DO (ppm)'] = O2solubilities * 32 * 1000 / test_matrix['Density @ 85C'] * test_matrix['Saturated with oxygen?']
test_matrix = test_matrix.drop(columns=['Saturated with oxygen?'])

dataset = []
for sheet in ['Time-Rf', 'Time-Rf2']:
  time_rf = pd.read_excel("drive/MyDrive/Asomaning, 1997.xlsx", sheet_name=sheet)
  runs = time_rf.columns.to_list()[1:]
  for run in runs:
    run_rf = time_rf[['Time', run]].dropna().to_numpy(dtype='float64')
    oper_data = test_matrix[test_matrix['Run #'] == run].iloc[0, :]
    oper_data_repeat = np.repeat([oper_data], np.shape(run_rf)[0], axis=0)
    dataset.append(np.concatenate((oper_data_repeat, run_rf), axis=1))

test_matrix_labels = test_matrix.columns.to_list()
dataset_with_run_num = pd.DataFrame(np.concatenate(dataset), columns=test_matrix_labels + ['Time', 'Rf'])
dataset = dataset_with_run_num.drop(columns=['Run #'])


Ensuring no N/A

In [ ]:
dataset.isna().sum().sum()

Maybe you want to see the dataset...

In [ ]:
dataset.shape

In [ ]:
dataset_with_run_num[dataset_with_run_num['Run #'] == 101]

Save the dataset as a csv file directly to My Drive

In [ ]:
dataset_with_run_num.to_csv("drive/MyDrive/dataset_with_run_num.csv", index=False)

In [ ]:
dataset.to_csv("drive/MyDrive/dataset.csv", index=False)